In [ ]:
# import allennlp
import torch
from transformers import AutoTokenizer, AutoModel
# import pandas as pd
# import json
# import torch
# from allennlp.modules.span_extractors.endpoint_span_extractor import EndpointSpanExtractor
# from tqdm import tqdm
# from utils.drqa.DocRanker import docranker_utils
# from utils.drqa.DocRanker.tokenizer import CoreNLPTokenizer
# from sentence_transformers import SentenceTransformer

: 

In [2]:
tfidf_path = "data-dir/test/sqlite_con-tfidf-ngram=3-hash=33554432-tokenizer=corenlp.npz"

In [3]:
_ , metadata = docranker_utils.load_sparse_csr(tfidf_path)

In [4]:
# write a function to read a json file line by line as a dictionary
def read_jsonl(file_path):
	with open(file_path, 'r') as f:
		data = []
		for line in f:
			data.append(json.loads(line))
	return data

test_data = read_jsonl("data-dir/test/contexts.json")

In [5]:
texts = []
for id in metadata['doc_dict'][1]:
	for point in test_data:
		if point['id'] == id:
			texts.append(point['text'])

In [3]:
model = SentenceTransformer('sentence-transformers/multi-qa-distilbert-cos-v1').to('cuda')

01/22/2023 03:59:22 PM: [ Load pretrained SentenceTransformer: sentence-transformers/multi-qa-distilbert-cos-v1 ]
01/22/2023 03:59:22 PM: [ Load pretrained SentenceTransformer: sentence-transformers/multi-qa-distilbert-cos-v1 ]


ImportError: 
AutoModel requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.


In [7]:
model.to('cuda')

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [8]:
embeddings = model.encode(texts, show_progress_bar=True, convert_to_tensor=True)

Batches: 100%|██████████| 273/273 [00:15<00:00, 17.88it/s]


In [10]:
# save embeddings
torch.save(embeddings, 'data-dir/test/sentence_transformer_embeddings_multi-qa-distilbert-cos-v1.pt')

In [9]:
embeddings.shape

torch.Size([8726, 768])

In [15]:
# normalize the embeddings
embeddings = embeddings / embeddings.norm(dim=1, keepdim=True)

In [12]:
# convert the embeddings to a numpy array
embeddings = embeddings.cpu().numpy()

In [13]:
import numpy as np


In [14]:
# save the embeddings
np.save("data-dir/test/sentence_transformer_embeddings_multi-qa-distilbert-cos-v1.npy", embeddings)

In [7]:
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased").to("cuda")

ImportError: 
AutoModel requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.


In [14]:
# create a function to tokenize this text and return its embeddings
def get_embeddings(texts):
	encoded_input = tokenizer(texts, padding=False, truncation=True, return_tensors='pt').to("cuda")
	with torch.no_grad():
		model_output = model(**encoded_input)
	return model_output.last_hidden_state

In [12]:
def return_cls_embedding(texts):
	return get_embeddings(texts)[:,0,:]

In [10]:
# create an endpoint span extractor
span_extractor = EndpointSpanExtractor(input_dim=768, combination="x,y")

In [11]:
# write a function to get the endpoint span embeddings using allennlp endpoint span extractor
def get_endpoint_span_embeddings(text):
	embeddings = get_embeddings(text)
	# extract the span from start till end, do it without gradient calculation
	with torch.no_grad():
		span_embeddings = span_extractor(embeddings, torch.tensor([[0, embeddings.shape[1]-1]]))
	return span_embeddings

In [15]:
# create a torch tensor to store the embeddings of all the texts using the get_endpoint_span_embeddings function
all_embeddings = []
for text in tqdm(texts):
	all_embeddings.append(return_cls_embedding(text))
all_embeddings = torch.cat(all_embeddings, dim=0)

# # load the saved span embeddings
# all_embeddings = torch.load("data-dir/test/endpoint_span_embeddings.pt")
import numpy as np
all_embeddings = all_embeddings.cpu().numpy()

100%|██████████| 8726/8726 [01:05<00:00, 133.27it/s]


In [20]:
# normalize the embeddings
all_embeddings = all_embeddings / np.linalg.norm(all_embeddings, axis=1, keepdims=True)

In [24]:

# save these embeddings as a numpy array
np.save("data-dir/test/cls_embeddings_normalised.npy", all_embeddings)

In [18]:
all_embeddings.shape

(8726, 1536)

In [6]:
import faiss

In [7]:
all_embeddings.shape

(8726, 1664)

In [8]:
d = 1664

In [9]:
index = faiss.IndexFlatL2(d) 

In [11]:
index.add(all_embeddings)

In [27]:
D, I = index.search(all_embeddings[0], 4)

ValueError: not enough values to unpack (expected 2, got 1)

In [28]:
all_embeddings[0].shape

(1664,)

In [29]:
all_embeddings[0:1].shape

(1, 1664)

In [21]:
all_embeddings[0]

array([-0.3770927 , -0.35221133, -0.33465827, ...,  0.039351  ,
       -0.07162095,  0.00525754], dtype=float32)

In [19]:
D.squeeze()

array([  0.     , 168.50912, 199.08183, 202.2987 ], dtype=float32)

In [24]:
I

array([  0,  38, 140, 144])

In [25]:
# create a random numpy array of size 145
random_array = np.random.randint(0, 145, 145)

In [26]:
random_array[I]

array([ 60,  10, 139,  65])